# NLP-10-1: Fake News Classifier
#### Credit:
https://www.youtube.com/watch?v=MO5n5JaRotc&list=PLZoTAELRMXVMdJ5sqbCK2LiM0HhQVWNzm&index=13

#### Import and read the fake news data, thanks to **kaggle**.
#### Credit: https://www.kaggle.com/competitions/fake-news/data#

## Fake News
Build a system to identify unreliable news articles

### Dataset Description
**train.csv :** A full training dataset with the following attributes:

- **id     :** unique id for a news article
- **title  :** the title of a news article
- **author :** author of the news article
- **text   :** the text of the article; could be incomplete
- **label  :** a label that marks the article as potentially unreliable
 - **1 :** unreliable
 - **0 :** reliable

**test.csv :** A testing training dataset with all the same attributes at <u>train.csv</u> without the label.

**submit.csv :** A sample submission that you can

In [3]:
import pandas as pd
import spacy
from spacy.lang.en import stop_words
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
import sys

## Adding a file to a repository on GitHub
Files that you add to a repository via a browser are limited to 25 MiB per file. You can add larger files, up to 100 MiB each, via the command line. For more information, see ["Adding a file to a repository using the command line."](https://docs.github.com/en/repositories/working-with-files/managing-files/adding-a-file-to-a-repository#adding-a-file-to-a-repository-using-the-command-line) To add files larger than 100 MiB, you must use Git Large File Storage. For more information, see ["About large files on GitHub."](https://docs.github.com/en/repositories/working-with-files/managing-large-files/about-large-files-on-github)

In [14]:
# from logging import raiseExceptions
# url = "https://raw.githubusercontent.com/akdubey2k/NLP/main/Fake_News_Classifier/train.csv"

# url = "fake_news_train.csv"
# df = pd.read_csv("fake_news_train.csv", sep=",", encoding='Latin-1')
# df = pd.read_csv("fake_news_train.csv", sep=",", encoding='latin1')
df = pd.read_csv("fake_news_train.csv", sep=",", encoding='utf-8')
'''
try:
  df = pd.read_csv(url, sep=",", encoding='Latin-1')
except FileNotFoundError:
  print(f"File {url} not found.  Aborting")
  raise
except OSError:
  print(f"OS error occurred trying to open {url}")
  raiseExceptions
except Exception as err:
  print(f"Unexpected error opening {url} is",repr(err))
  raise
except UnicodeDecodeError:
  df = pd.read_csv(url, sep=",", encoding="latin1")

if df is None:
  df = pd.read_csv(url, sep=',', encoding="utf-8")
'''
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


#### Check the **shape** of dataframe

In [19]:
# shape of entire dataset
print("df.shape".ljust(50, '.'), df.shape)

df.shape.......................................... (20800, 5)


#### Segregate data, as **dependent and independent feature**.

In [20]:
X = df.drop('label', axis=1)  # independent feature
print("X.shape".ljust(50, '.'), X.shape)
X.head(2)

X.shape........................................... (20800, 4)


,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...


In [26]:
y = df['label']
print("y.shape".ljust(50, '.'), y.shape)
y.head(2)

y.shape........................................... (20800,)


0    1
1    0
Name: label, dtype: int64

## Check the null values in the dataset

In [22]:
# Summarize the number of na in every feature column dataset
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

## Replacing the null values with empty string

In [23]:
df = df.fillna(' ')
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [25]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Load English tokenizer, tagger, parser, NER and word vectors

In [ ]:
spacy_nlp = spacy.load('en_core_web_sm')
spacy_stopwords = stop_words.STOP_WORDS
stopwords = English()
stopwords('hello I am english')

In [11]:
len(spacy_stopwords)

326

# Text Pre-processing